# User IDs
- every user on \*nix system has unique user ID number
- **id** command can be used to display user's id

In [1]:
! id

uid=1001(user) gid=1001(user) groups=1001(user),27(sudo)


In [2]:
! id -u

1001


In [3]:
! id root

uid=0(root) gid=0(root) groups=0(root)


## add 2 users to the system
- add tom and jerry as users to play with notesearch program provided by Hacking book
    - `$ sudo adduser tom`
    - `$ sudo adduser jerry`

In [4]:
! id tom

uid=1003(tom) gid=1003(tom) groups=1003(tom)


In [5]:
! id jerry

id: ‘jerry’: no such user


In [6]:
! finger tom

Login: tom            			Name: Tom Cat
Directory: /home/tom                	Shell: /bin/bash
Office: 123, x3132			Home Phone: 23-2323
Never logged in.
No mail.
No Plan.


## Setuid programs
- some programs need to be accessed by all users
- all users need to access and modify common files like /etc/passwd, /etc/shadow
- how's that possible?
    - **setuid** flag to the rescue
- **setuid** programs have **s** in the ls -l output
    - e.g., see passwd and chsh programs
    - these programs are owned by root
    - run as root user when any user run these programs
    - the logic is written in such a way that only the line pertaining to the user can be modified by each user
- running program can have both a real user ID and effective user ID
    - can be retrieved using getuid() and geteuid() respectively
    - see booksrc/uid_demo.c

In [14]:
# password file with user info
! ls -l /etc/passwd /etc/shadow

-rw-r--r-- 1 root root   3441 Mar 11 00:51 /etc/passwd
-rw-r----- 1 root shadow 2196 Mar 11 00:51 /etc/shadow


In [15]:
# password and chsh programs
! ls -l $(which passwd)  $(which chsh)

-rwsr-xr-x 1 root root 47780 Jul 27  2018 /usr/bin/chsh
-rwsr-xr-x 1 root root 62024 Jul 27  2018 /usr/bin/passwd


### booksrc/uid_demo.c

In [19]:
! cat ./booksrc/uid_demo.c

#include <stdio.h>
#include <unistd.h>
#include <sys/types.h>

int main()
{
   printf("real uid: %d\n", getuid());
   printf("effective uid: %d\n", geteuid());
}


In [20]:
%%bash
in=./booksrc/uid_demo.c
out=uid_demo.out
gcc -o $out $in

In [21]:
! ./uid_demo.out

real uid: 1001
effective uid: 1001


In [22]:
! ls -l uid_demo.out

-rwxr-xr-x 1 user user 15524 Mar 11 02:27 uid_demo.out


Modify the ownership to root from terminal as it requires sudo priviledge

`sudo chown root:root ./uid_demo`

In [23]:
%%bash
ls -l uid_demo.out
./uid_demo.out

-rwxr-xr-x 1 user user 15524 Mar 11 02:27 uid_demo.out
real uid: 1001
effective uid: 1001


Set setuid flag using terminal

`sudo chmod u+s ./uid_demo`

In [24]:
%%bash
ls -l uid_demo.out
./uid_demo.out

-rwxr-xr-x 1 user user 15524 Mar 11 02:27 uid_demo.out
real uid: 1001
effective uid: 1001


## compile simplenote.c in hacking/booksrc
- add several notes for the loggedin user
- try adding notes for other users
    - how do we fix the problem?
    - better program... booksrc/notetaker.c

In [25]:
! gcc -o simplenote.out ./booksrc/simplenote.c

./booksrc/simplenote.c: In function ‘main’:
./booksrc/simplenote.c:35:7: warning: assignment to ‘FILE *’ {aka ‘struct _IO_FILE *’} from ‘int’ makes pointer from integer without a cast [-Wint-conversion]
    fd = open(datafile, O_WRONLY|O_CREAT|O_APPEND, S_IRUSR|S_IWUSR);
       ^
./booksrc/simplenote.c:40:13: warning: passing argument 1 of ‘write’ makes integer from pointer without a cast [-Wint-conversion]
    if(write(fd, buffer, strlen(buffer)) == -1)
             ^~
In file included from ./booksrc/simplenote.c:6:
/usr/include/unistd.h:366:27: note: expected ‘int’ but argument is of type ‘FILE *’ {aka ‘struct _IO_FILE *’}
 extern ssize_t write (int __fd, const void *__buf, size_t __n) __wur;
                       ~~~~^~~~
./booksrc/simplenote.c:43:13: warning: passing argument 1 of ‘close’ makes integer from pointer without a cast [-Wint-conversion]
    if(close(fd) == NULL)
             ^~
In file included from ./booksrc/simplenote.c:6:
/usr/include/unistd.h:353:23: note: expected

In [17]:
! ls -al /tmp/notes

-rw------- 1 user user 38 Mar 11 01:59 /tmp/notes


## compile notetaker.c in hacking/booksrc
- set setuid
- add several notes for each user
- notes are written to /var/notes instead of /tmp/notes